# Notebook 10: Heuristic Validation

## Purpose
Validate that scoring heuristics (nutrition, security, etc.) align with actual elk behavior and are predictive of presence.

## Key Questions
- Which heuristic scores are available in the dataset?
- Do heuristic scores correlate with their source features?
- Are heuristics discriminative for elk presence?
- Do composite scores outperform individual heuristics?
- Should heuristics be kept as features or replaced with raw features?

## Key Observations to Look For
- **Score-Feature Correlation**: Heuristics should correlate with source features (>0.5)
- **Discriminative Power**: AUC-ROC > 0.5 (better than random)
- **Composite Performance**: Should composite score improve on individual scores?
- **Weight Validation**: Do heuristic weights align with data-driven importance?


In [ ]:
# Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.metrics import roc_curve, auc, confusion_matrix, precision_score, recall_score, f1_score
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set random seed
np.random.seed(42)

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# Determine project root and output directories
possible_roots = [
    Path('.'),  # If running from project root
    Path('..'),  # If running from notebooks directory
    Path('../..'),  # If running from subdirectory
]

data_root = None
for root in possible_roots:
    if (root / 'data' / 'features').exists():
        data_root = root / 'data'
        break

if data_root is None:
    data_root = Path('../data')

# Create output directories relative to project root
figures_dir = data_root / 'figures'
reports_dir = data_root / 'reports'
figures_dir.mkdir(parents=True, exist_ok=True)
reports_dir.mkdir(parents=True, exist_ok=True)

print(f'✓ Setup complete')
print(f'  Output directory: {data_root.absolute()}')


## 1. Load Data and Identify Heuristic Scores

Heuristic scores are expert-derived features that combine multiple raw features into ecologically meaningful composite measures (e.g., nutrition score from NDVI, security score from cover and roads). This section identifies which heuristic scores are available in the dataset.

### What to Look For
- Confirm expected heuristic columns are present (nutrition_score, security_score, etc.)
- Note range of scores (typically 0-1 or 0-100)
- Check for missing values in heuristic columns

In [ ]:
# Load data
from pathlib import Path

# Try multiple possible paths
possible_paths = [
    Path('data/features/complete_context.csv'),  # From project root
    Path('../data/features/complete_context.csv'),  # From notebooks directory
    Path('../../data/features/complete_context.csv'),  # From subdirectory
]

data_path = None
for path in possible_paths:
    if path.exists():
        data_path = path
        break

if data_path is None:
    raise FileNotFoundError(
        f'Data file not found. Tried: {[str(p) for p in possible_paths]}\n'
        f'Please run: python scripts/combine_feature_files.py\n'
        f'Or ensure you are running the notebook from the project root directory.'
    )

print(f'Loading data from: {data_path}')
df = pd.read_csv(data_path)

# Automatically detect heuristic score columns
heuristic_cols = [col for col in df.columns if 'score' in col.lower()]

print(f'Dataset shape: {df.shape}')
print(f'\nFound {len(heuristic_cols)} heuristic score columns:')
for col in heuristic_cols:
    print(f'  - {col}')

if len(heuristic_cols) == 0:
    print('\n⚠ No heuristic score columns found. Skipping heuristic validation.')
    print('This notebook requires columns containing "score" in the name.')
else:
    # Detect presence column
    presence_col = None
    for col in df.columns:
        if col.lower() in ['presence', 'target', 'label', 'is_presence', 'elk_present']:
            presence_col = col
            break
    
    print(f'\nPresence column: {presence_col}')
    
    # Display heuristic score summary statistics
    print('\nHeuristic score statistics:')
    print(df[heuristic_cols].describe())


## 2. Heuristic Score Distributions

Before validating heuristics against outcomes, we examine their distributions to ensure they vary appropriately and don't have pathological characteristics (constant values, extreme skew).

### What This Code Does
- Creates histograms for each heuristic score
- Overlays mean and median reference lines
- Compares score distributions between presence and absence classes

### Interpreting Heuristic Distributions

**Expected Characteristics:**
- **Range**: Scores should span most of their theoretical range (e.g., 0-100)
- **Variation**: Standard deviation should indicate meaningful spread
- **Shape**: Roughly normal or uniform; avoid extreme spikes at bounds

**Red Flags:**
- **Constant value**: std ≈ 0 indicates broken heuristic calculation
- **Spike at 0 or max**: May indicate ceiling/floor effects or missing data handling
- **Bimodal**: May be appropriate (winter/summer) or indicate issues

### Presence vs Absence Comparison

**Expected Pattern:**
- **Presence mean > Absence mean**: Heuristics should favor presence locations
- **Substantial difference**: At least 5-10% of scale difference
- **Overlapping distributions**: Some overlap is expected (not perfectly discriminative)

### What to Look For
- **Nutrition score**: Presence should have higher mean (better forage)
- **Security score**: Presence should have higher mean (more cover)
- **Water score**: Presence should have higher mean (closer to water)
- **Overall/composite score**: Presence should be substantially higher

In [ ]:
# Create histograms for all heuristic scores
if len(heuristic_cols) > 0:
    n_cols = 3
    n_rows = (len(heuristic_cols) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
    axes = axes.flatten() if n_rows > 1 else [axes] if n_cols == 1 else axes
    
    for idx, col in enumerate(heuristic_cols):
        ax = axes[idx]
        data = df[col].dropna()
        
        if len(data) > 0:
            # Histogram
            ax.hist(data, bins=50, alpha=0.7, color='steelblue', edgecolor='black')
            
            # Add mean/median lines
            mean_val = data.mean()
            median_val = data.median()
            ax.axvline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_val:.2f}')
            ax.axvline(median_val, color='blue', linestyle='--', linewidth=2, label=f'Median: {median_val:.2f}')
            
            ax.set_xlabel(col, fontsize=10)
            ax.set_ylabel('Frequency', fontsize=10)
            ax.set_title(col, fontsize=11)
            ax.legend(fontsize=8)
            ax.grid(alpha=0.3)
    
    # Hide extra subplots
    for idx in range(len(heuristic_cols), len(axes)):
        axes[idx].axis('off')
    
    plt.suptitle('Heuristic Score Distributions', fontsize=16, y=1.00)
    plt.tight_layout()
    plt.savefig(figures_dir / 'heuristic_distributions.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print('✓ Saved heuristic distributions')
    
    # Compare presence vs absence if available
    if presence_col:
        print('\n### Presence vs Absence Comparison:')
        for col in heuristic_cols:
            presence_mean = df[df[presence_col] == 1][col].mean()
            absence_mean = df[df[presence_col] == 0][col].mean()
            diff = presence_mean - absence_mean
            print(f'  {col}: Presence={presence_mean:.3f}, Absence={absence_mean:.3f}, Diff={diff:.3f}')
else:
    print('⚠ No heuristic scores to analyze')


## 3. Score vs Source Features Validation

Each heuristic score is computed from underlying "source" features (e.g., nutrition_score from NDVI). This section validates that scores correlate appropriately with their source features, confirming the heuristic calculation is working correctly.

### What This Code Does
- Maps each heuristic to its expected source features
- Creates scatter plots of score vs source feature
- Calculates correlation between score and source
- Flags low correlations as validation failures

### Interpreting Validation Results

**Expected Correlations:**
- **r > 0.7**: Strong correlation - heuristic accurately reflects source
- **r 0.5-0.7**: Moderate correlation - heuristic combines multiple factors
- **r < 0.5**: Weak correlation - may indicate calculation issues

**Score-Source Mappings:**
| Heuristic | Expected Source Features |
|-----------|-------------------------|
| nutrition_score | ndvi, vegetation quality |
| security_score | canopy_cover, road_distance |
| water_score | water_distance |
| elevation_score | elevation (and slope) |
| snow_score | snow_depth, snow_water_equiv |

### What to Look For
- **Linear relationship**: Score increases/decreases predictably with source
- **Non-linear transformations**: Curved relationships indicate non-linear scoring functions
- **Outliers**: Points far from the trend may indicate edge cases in scoring logic
- **Ceiling effects**: Flat regions at score extremes

### Validation Failures
If score-source correlation is low (<0.5):
1. **Check heuristic calculation**: May have bugs or incorrect weights
2. **Check source data quality**: Source feature may have placeholder values
3. **Check feature mapping**: Source feature name may have changed
4. **Consider multi-factor scoring**: Score may legitimately combine multiple sources

In [ ]:
# Validate heuristic scores against their source features
if len(heuristic_cols) > 0:
    # Map heuristic names to likely source features
    source_mapping = {}
    for score_col in heuristic_cols:
        score_lower = score_col.lower()
        source_features = []
        
        # Try to identify source features based on heuristic name
        for col in df.columns:
            col_lower = col.lower()
            if col == score_col:
                continue
            
            # Nutrition score -> NDVI
            if 'nutrition' in score_lower and 'ndvi' in col_lower:
                source_features.append(col)
            # Water score -> water_distance
            elif 'water' in score_lower and 'water' in col_lower and 'distance' in col_lower:
                source_features.append(col)
            # Elevation score -> elevation
            elif 'elevation' in score_lower and ('elev' in col_lower or 'altitude' in col_lower):
                source_features.append(col)
            # Security score -> security_habitat or road_distance
            elif 'security' in score_lower and ('security' in col_lower or 'road' in col_lower):
                source_features.append(col)
            # Snow score -> snow_depth or snow_water_equiv
            elif 'snow' in score_lower and 'snow' in col_lower:
                source_features.append(col)
        
        if source_features:
            source_mapping[score_col] = source_features
    
    # Create scatter plots for score vs source features
    if len(source_mapping) > 0:
        n_pairs = sum(len(sources) for sources in source_mapping.values())
        n_cols = 3
        n_rows = (n_pairs + n_cols - 1) // n_cols
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
        axes = axes.flatten() if n_rows > 1 else [axes] if n_cols == 1 else axes
        
        plot_idx = 0
        validation_results = []
        
        for score_col, source_features in source_mapping.items():
            for source_col in source_features:
                ax = axes[plot_idx]
                
                # Sample if too many points
                sample_size = min(5000, len(df))
                df_sample = df.sample(n=sample_size, random_state=42)
                
                # Scatter plot
                ax.scatter(df_sample[source_col], df_sample[score_col], 
                          alpha=0.3, s=10, edgecolors='none')
                
                # Calculate correlation
                corr = df[[source_col, score_col]].corr().iloc[0, 1]
                
                # Add regression line
                z = np.polyfit(df_sample[source_col].dropna(), 
                              df_sample[score_col].dropna(), 1)
                p = np.poly1d(z)
                x_line = np.linspace(df_sample[source_col].min(), 
                                    df_sample[source_col].max(), 100)
                ax.plot(x_line, p(x_line), "r--", alpha=0.8, linewidth=2)
                
                ax.set_xlabel(source_col, fontsize=10)
                ax.set_ylabel(score_col, fontsize=10)
                ax.set_title(f'{score_col} vs {source_col}\nr={corr:.3f}', fontsize=11)
                ax.grid(alpha=0.3)
                
                validation_results.append({
                    'heuristic': score_col,
                    'source_feature': source_col,
                    'correlation': corr,
                    'status': 'PASS' if abs(corr) > 0.5 else 'FAIL'
                })
                
                plot_idx += 1
        
        # Hide extra subplots
        for idx in range(plot_idx, len(axes)):
            axes[idx].axis('off')
        
        plt.suptitle('Heuristic Score Validation (vs Source Features)', fontsize=16, y=1.00)
        plt.tight_layout()
        plt.savefig(figures_dir / 'heuristic_validation.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        print('✓ Saved heuristic validation plots')
        
        # Print validation results
        validation_df = pd.DataFrame(validation_results)
        print('\n### Validation Results:')
        print(validation_df)
        
        failed = validation_df[validation_df['status'] == 'FAIL']
        if len(failed) > 0:
            print(f'\n⚠ WARNING: {len(failed)} heuristic-source pairs have low correlation (<0.5):')
            for _, row in failed.iterrows():
                print(f"  - {row['heuristic']} vs {row['source_feature']}: r={row['correlation']:.3f}")
        else:
            print('\n✓ All heuristic-source correlations meet threshold (>0.5)')
    else:
        print('⚠ Could not identify source features for heuristics')
else:
    print('⚠ No heuristic scores to validate')


## 4. Heuristic Discriminative Power (ROC Analysis)

The ultimate test of a heuristic is whether it can discriminate between presence and absence. ROC (Receiver Operating Characteristic) curves and AUC (Area Under Curve) quantify this discriminative ability.

### What This Code Does
- Calculates ROC curves treating each heuristic as a classifier
- Computes AUC-ROC for each heuristic
- Compares heuristics against random baseline (AUC = 0.5)
- Creates a bar chart comparing AUC across heuristics

### Understanding ROC Curves

**ROC Curve Interpretation:**
- **X-axis**: False Positive Rate (1 - Specificity)
- **Y-axis**: True Positive Rate (Sensitivity/Recall)
- **Diagonal line**: Random classifier (no discrimination)
- **Curve above diagonal**: Better than random
- **Perfect classifier**: Goes to top-left corner (AUC = 1.0)

**AUC Interpretation:**
- **AUC = 0.5**: No better than random guessing
- **AUC 0.5-0.6**: Poor discrimination
- **AUC 0.6-0.7**: Acceptable discrimination
- **AUC 0.7-0.8**: Good discrimination
- **AUC > 0.8**: Excellent discrimination

### What to Look For
- **All heuristics above diagonal**: Basic validation that they work
- **AUC ranking**: Which heuristics are most discriminative?
- **AUC < 0.55**: Heuristic is essentially useless - consider dropping
- **Composite vs individual**: Does combining scores improve AUC?

### Elk Ecology Context
Expected heuristic ranking (roughly):
1. **Composite/overall score**: Should be highest (combines all factors)
2. **Nutrition score**: Forage is primary driver of elk distribution
3. **Security score**: Cover and human avoidance are important
4. **Water score**: Water proximity matters, especially in summer
5. **Seasonal scores**: May vary by time of year

### Implications for Modeling
- **High AUC heuristics**: Keep as features or use directly for inference
- **Low AUC heuristics**: Drop or investigate why they fail
- **Similar AUC**: May indicate redundancy - keep most interpretable

In [ ]:
# Calculate AUC-ROC for each heuristic
if len(heuristic_cols) > 0 and presence_col:
    auc_results = []
    
    plt.figure(figsize=(12, 8))
    
    for col in heuristic_cols:
        # Remove NaN values
        valid_mask = df[[col, presence_col]].notna().all(axis=1)
        y_true = df.loc[valid_mask, presence_col]
        y_scores = df.loc[valid_mask, col]
        
        if len(y_true) > 0 and len(y_scores) > 0:
            # Calculate ROC curve
            fpr, tpr, thresholds = roc_curve(y_true, y_scores)
            roc_auc = auc(fpr, tpr)
            
            auc_results.append({
                'heuristic': col,
                'auc': roc_auc
            })
            
            # Plot ROC curve
            plt.plot(fpr, tpr, linewidth=2, label=f'{col} (AUC = {roc_auc:.3f})')
    
    # Add diagonal line (random classifier)
    plt.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random (AUC = 0.5)')
    
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate', fontsize=12)
    plt.title('ROC Curves for Heuristic Scores', fontsize=14, pad=20)
    plt.legend(loc='lower right', fontsize=10)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig(figures_dir / 'heuristic_roc_curves.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print('✓ Saved ROC curves')
    
    # Create AUC comparison bar chart
    if len(auc_results) > 0:
        auc_df = pd.DataFrame(auc_results).sort_values('auc', ascending=False)
        
        plt.figure(figsize=(12, 6))
        colors = ['green' if x > 0.5 else 'red' for x in auc_df['auc']]
        plt.barh(range(len(auc_df)), auc_df['auc'], color=colors, alpha=0.7, edgecolor='black')
        plt.axvline(0.5, color='red', linestyle='--', linewidth=2, label='Random (0.5)')
        plt.yticks(range(len(auc_df)), auc_df['heuristic'])
        plt.xlabel('AUC-ROC', fontsize=12)
        plt.ylabel('Heuristic Score', fontsize=12)
        plt.title('Heuristic Discriminative Power (AUC-ROC)', fontsize=14, pad=20)
        plt.legend()
        plt.grid(axis='x', alpha=0.3)
        plt.tight_layout()
        plt.savefig(figures_dir / 'heuristic_auc_comparison.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        print('\n### AUC-ROC Results (sorted by AUC):')
        print(auc_df)
        
        # Flag poor performers
        poor_performers = auc_df[auc_df['auc'] < 0.55]
        if len(poor_performers) > 0:
            print(f'\n⚠ WARNING: {len(poor_performers)} heuristics perform no better than random (AUC < 0.55):')
            for _, row in poor_performers.iterrows():
                print(f"  - {row['heuristic']}: AUC={row['auc']:.3f}")
        else:
            print('\n✓ All heuristics perform better than random')
else:
    print('⚠ Cannot calculate AUC without heuristic scores and presence column')


## 5. Confusion Matrix for Top Heuristic

To understand how well the best heuristic performs as an actual classifier, we convert it to binary predictions using a threshold and examine the confusion matrix.

### What This Code Does
- Selects the heuristic with highest AUC
- Uses median score as classification threshold
- Generates predictions (above median = predict presence)
- Creates confusion matrix and calculates performance metrics

### Interpreting the Confusion Matrix

```
                  Predicted
                  Absence  Presence
Actual  Absence    TN        FP
        Presence   FN        TP
```

- **True Negatives (TN)**: Correctly predicted absence
- **False Positives (FP)**: Predicted presence but was absence (Type I error)
- **False Negatives (FN)**: Predicted absence but was presence (Type II error)
- **True Positives (TP)**: Correctly predicted presence

### Performance Metrics

- **Accuracy**: (TP + TN) / Total - overall correctness
- **Precision**: TP / (TP + FP) - of predicted presence, how many correct?
- **Recall**: TP / (TP + FN) - of actual presence, how many found?
- **F1-Score**: Harmonic mean of precision and recall

### What to Look For
- **Accuracy vs baseline**: Must exceed majority class rate
- **Precision/Recall balance**: Trade-off depends on application
- **Diagonal dominance**: Most values on diagonal = good classifier
- **Error patterns**: Where does the heuristic fail?

### Ecological Interpretation of Errors
- **False Positives**: Heuristic says good habitat, but no elk - may be unused suitable habitat
- **False Negatives**: Elk present but heuristic says poor habitat - heuristic missing important factors

### Threshold Considerations
Using median as threshold is arbitrary. For inference applications:
- **Higher threshold**: Fewer but more confident predictions
- **Lower threshold**: More predictions but higher false positive rate
- **Optimize for use case**: Conservation may want high recall; hunting management may want high precision

In [ ]:
# Create confusion matrix for best heuristic
if len(heuristic_cols) > 0 and presence_col and len(auc_results) > 0:
    # Get best heuristic
    best_heuristic = auc_df.iloc[0]['heuristic']
    best_auc = auc_df.iloc[0]['auc']
    
    print(f'Using best heuristic: {best_heuristic} (AUC={best_auc:.3f})')
    
    # Create binary classifier using median threshold
    valid_mask = df[[best_heuristic, presence_col]].notna().all(axis=1)
    y_true = df.loc[valid_mask, presence_col]
    y_scores = df.loc[valid_mask, best_heuristic]
    
    threshold = y_scores.median()
    y_pred = (y_scores > threshold).astype(int)
    
    # Calculate confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Calculate metrics
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    accuracy = (y_pred == y_true).mean()
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True,
                xticklabels=['Absence', 'Presence'],
                yticklabels=['Absence', 'Presence'])
    plt.xlabel('Predicted', fontsize=12)
    plt.ylabel('Actual', fontsize=12)
    plt.title(f'Confusion Matrix: {best_heuristic}\n'
              f'Threshold={threshold:.3f}, Accuracy={accuracy:.3f}, F1={f1:.3f}', 
              fontsize=14, pad=20)
    plt.tight_layout()
    plt.savefig(figures_dir / 'heuristic_confusion_matrix.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f'\n### Performance Metrics:')
    print(f'  Accuracy: {accuracy:.3f}')
    print(f'  Precision: {precision:.3f}')
    print(f'  Recall: {recall:.3f}')
    print(f'  F1-Score: {f1:.3f}')
    print(f'  Baseline (always majority): {(y_true == y_true.mode()[0]).mean():.3f}')
    
    if accuracy > (y_true == y_true.mode()[0]).mean():
        print('\n✓ Heuristic outperforms majority class baseline')
    else:
        print('\n⚠ Heuristic does not outperform majority class baseline')
else:
    print('⚠ Cannot create confusion matrix without heuristic scores and presence column')


## 6. Heuristic Recommendations

Based on all validation analyses, this section generates actionable recommendations for each heuristic: keep, drop, or review/recalibrate.

### What This Code Does
- Combines AUC and source correlation results
- Applies decision rules to generate recommendations
- Creates a summary report saved to `data/reports/heuristic_recommendations.md`

### Recommendation Categories

**KEEP (AUC > 0.55, good source correlation):**
- Heuristic is working as intended
- Include in feature set for modeling
- May use directly for inference

**REVIEW (mixed results):**
- Low source correlation but reasonable AUC: Check calculation logic
- Good source correlation but low AUC: Source feature may not be predictive
- Investigate before including in models

**DROP (AUC < 0.55):**
- Heuristic provides no discrimination
- Likely to add noise rather than signal
- Consider replacing with raw features

### How to Use Recommendations

**For Model Training:**
1. Include KEEP heuristics as features
2. Investigate REVIEW heuristics before including
3. Exclude DROP heuristics

**For Heuristic Refinement:**
1. Review weighting schemes for low-performing heuristics
2. Consider adding missing factors
3. Validate against ecological literature

**For Inference Pipeline:**
1. KEEP heuristics can be used for real-time scoring
2. DROP heuristics should be removed from inference
3. REVIEW heuristics need fixes before production use

### Saved Output
The detailed recommendations are saved to `data/reports/heuristic_recommendations.md` for reference during model development and heuristic refinement.

In [ ]:
# Generate recommendations
if len(heuristic_cols) > 0:
    recommendations = []
    
    # Compile all analysis results
    if 'auc_df' in locals():
        for _, row in auc_df.iterrows():
            heuristic = row['heuristic']
            auc_val = row['auc']
            
            rec = {
                'heuristic': heuristic,
                'auc': auc_val,
                'recommendation': 'KEEP' if auc_val > 0.55 else 'DROP',
                'reason': f'AUC={auc_val:.3f} {"above" if auc_val > 0.55 else "below"} 0.55 threshold'
            }
            
            # Check validation if available
            if 'validation_df' in locals():
                validation = validation_df[validation_df['heuristic'] == heuristic]
                if len(validation) > 0:
                    low_corr = validation[validation['status'] == 'FAIL']
                    if len(low_corr) > 0:
                        rec['recommendation'] = 'REVIEW'
                        rec['reason'] += f'; Low correlation with source features'
            
            recommendations.append(rec)
    
    if len(recommendations) > 0:
        rec_df = pd.DataFrame(recommendations)
        
        print('### Heuristic Recommendations:')
        print(rec_df)
        
        # Save recommendations
        report = f'''# Heuristic Validation Recommendations

Generated: {pd.Timestamp.now()}

## Summary

Total heuristics analyzed: {len(heuristic_cols)}

## Recommendations

'''
        
        for _, rec in rec_df.iterrows():
            report += f'''
### {rec['heuristic']}
- **AUC-ROC**: {rec['auc']:.3f}
- **Recommendation**: {rec['recommendation']}
- **Reason**: {rec['reason']}

'''
        
        report += '''
## General Guidelines

1. **KEEP**: Heuristics with AUC > 0.55 and good source feature correlation
2. **REVIEW**: Heuristics with low correlation to source features (may need recalibration)
3. **DROP**: Heuristics with AUC < 0.55 (no better than random)

## Next Steps

- Consider keeping high-performing heuristics as features
- Review and potentially recalibrate heuristics with low source correlation
- Replace low-performing heuristics with raw features in modeling
'''
        
        with open(reports_dir / 'heuristic_recommendations.md', 'w') as f:
            f.write(report)
        
        print('\n✓ Saved recommendations to data/reports/heuristic_recommendations.md')
    else:
        print('⚠ Could not generate recommendations')
else:
    print('⚠ No heuristics to recommend on')


## Summary

This notebook validated heuristic scores across multiple dimensions:

1. **Score Distributions**: Examined distribution of each heuristic
2. **Source Feature Validation**: Checked correlation with source features
3. **Discriminative Power**: Calculated AUC-ROC for each heuristic
4. **Confusion Matrix**: Evaluated best heuristic as binary classifier
5. **Recommendations**: Generated actionable recommendations

**Key Findings**:
- Review `data/reports/heuristic_recommendations.md` for detailed recommendations
- Heuristics with AUC > 0.55 are potentially useful features
- Heuristics with low source correlation may need recalibration

**Next Steps**:
- Proceed to Notebook 11 for target variable analysis
- Consider keeping high-performing heuristics as model features
- Review and fix heuristics with validation issues
